In [1]:
from wdtools import *

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# this will take about 17 minutes
start = time.time()
all_taxlot = combine_taxlot()
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 17.0 minutes to complete


In [3]:
df1 = combined_reindexed_data(setID='Set001', nm_to_add=0) 

In [6]:
res1 = check_corrected_data(df=df1, setID='Set001', all_taxlot=all_taxlot, nm_to_add=0, export=False)

In [7]:
len(res1[0].record_ID.unique())

1331

In [8]:
len(res1[0][~res1[0].wdID.isin(res1[1].wetdet_delin_number.values)].record_ID.unique())

1307

In [9]:
df_wlots_to_check1 = res1[3]

In [10]:
len(df_wlots_to_check1[df_wlots_to_check1.missinglot=='N'].record_ID.unique())

27

In [11]:
df_wlots_to_check1.to_csv(os.path.join(inpath + '\\output\\to_review\\', 'df_with_lots_check_Set001.csv'), index=False) 

In [12]:
df2 = combined_reindexed_data(setID='Set002', nm_to_add=1420) 

In [13]:
res2 = check_corrected_data(df=df2, setID='Set002', all_taxlot=all_taxlot, nm_to_add=1420, export=False)

In [14]:
df_wlots_to_check2 = res2[3]

In [15]:
df_wlots_to_check2.to_csv(os.path.join(inpath + '\\output\\to_review\\', 'df_with_lots_check_Set002.csv'), index=False) 

In [16]:
len(res2[0].record_ID.unique())

607

In [17]:
len(res2[0][res2[0].wdID.isin(res2[1].wetdet_delin_number.values)].record_ID.unique())

20

In [18]:
len(df_wlots_to_check2[df_wlots_to_check2.missinglot=='N'].record_ID.unique())

18

In [19]:
setgdf1 = gpd.read_file(os.path.join(inpath, 'GIS', 'Join_Statewide.gdb'), layer=f'WD_Set001_Combined')

In [20]:
setgdf2 = gpd.read_file(os.path.join(inpath, 'GIS', 'Join_Statewide.gdb'), layer=f'WD_Set002_Combined')

In [21]:
len(setgdf1.Record_ID.unique())

1362

In [22]:
missedmatch1 = compare_data_report(gdf=gpd.GeoDataFrame(res1[0], crs="EPSG:2992", geometry='geometry'), 
                                   setID='Set001', 
                                   nm_to_add=0, 
                                   export = True)

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\geodataframe.py:1299: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)
C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [23]:
len(missedmatch1[0])

23

In [26]:
len(missedmatch1[2])

54

In [24]:
missedmatch2 = compare_data_report(gdf=gpd.GeoDataFrame(res2[0], crs="EPSG:2992", geometry='geometry'), 
                                   setID='Set002', 
                                   nm_to_add=1420, 
                                   export = True)

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [25]:
len(missedmatch2[0])

4

In [27]:
len(missedmatch2[2])

15